In [2]:
import pandas as pd
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy import create_engine

In [3]:
gis_url = #copy from .env
analysis_url =  #copy from .env

ENGINE = create_engine(analysis_url)
GIS_ENGINE = create_engine(gis_url)

In [4]:
sa_name = "hunting_park"

In [5]:
Q_person_view_nonpdo = fr"""
CREATE OR REPLACE VIEW person_vehicle_nonpdo AS(
	with tblA AS(	
		select 
			p.crn,
			p.unit_num,
			p.person_type,
			p.inj_severity,
			v.unit_type,
			concat(text(p.crn), text(p.unit_num)) as joinid
		from person_data  p
		left join vehicle_data  v 
		on concat(text(p.crn), text(p.unit_num)) = concat(text(v.crn), text(v.unit_num))
		where not(p.inj_severity = '0') 
		)
	select 
		a.*
	from tblA a
	right join {sa_name}_crashes b
	on a.crn = b.crn   
	);
"""

Q_person_view_ksi = fr"""
CREATE OR REPLACE VIEW person_vehicle_ksi AS(
    with tblA AS(	
		select 
			p.crn,
			p.unit_num,
			p.person_type,
			p.inj_severity,
			v.unit_type,
			concat(text(p.crn), text(p.unit_num)) as joinid
		from person_data  p
		left join vehicle_data  v 
		on concat(text(p.crn), text(p.unit_num)) = concat(text(v.crn), text(v.unit_num))
		where p.inj_severity = '1' or p.inj_severity ='2'
	)
	select 
		a.*
	from tblA a
	right join {sa_name}_crashes b
	on a.crn = b.crn   
);
"""
ENGINE.execute(Q_person_view_nonpdo)
ENGINE.execute(Q_person_view_ksi)


In [6]:
#function to run queries to could specific types of crashes
def calculate(view_name):
    #bike
    q_bike = fr"""
        select count(*)
        from {view_name}
        where unit_type = '11'
        and person_type = '1';
    """
    bike = pd.read_sql(
        q_bike,
        con = ENGINE
    )

    bike['type'] = 'bike'

    #pedestrians
    q_ped = fr"""
        select count(*)
        from {view_name}
        where person_type = '7';
    """
    ped = pd.read_sql(
        q_ped,
        con = ENGINE
    )

    ped['type'] = 'ped'

    #vehicle occupants
    q_veh = fr"""
        select count(*)
        from {view_name}
        where (person_type = '1' or person_type = '2')
        and not(unit_type = '11');
    """
    veh = pd.read_sql(
        q_veh,
        con = ENGINE
    )
    veh['type'] = 'veh'

    #other
    q_other = fr"""
        select count(*)
        from {view_name}
        where person_type in ('8','9');
    """
    other = pd.read_sql(
        q_other,
        con = ENGINE
    )
    other['type'] = 'other'

    df = pd.concat([bike, ped, veh, other], ignore_index = True)
    return df



In [7]:
#Non-pdo sumamry
calculate('person_vehicle_nonpdo')


,count,type
0,5,bike
1,73,ped
2,299,veh
3,8,other


In [8]:
#KSI summary
calculate('person_vehicle_ksi')

,count,type
0,1,bike
1,15,ped
2,9,veh
3,0,other
